In [1]:
import pandas as pd
import seaborn as sns
import json
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import math

pd.options.display.max_columns = 100
pd.options.display.max_rows = 500

In [2]:
# STEP 1. Creating dataframes
tr_df = pd.read_csv("data/train.csv")
tst_df = pd.read_csv("data/test.csv")
task_df = pd.read_csv('data/task.csv')
taskres_df = pd.read_csv('data/task_results.csv')
course_df = pd.read_csv('data/course.csv')

In [3]:
# STEP 2. Identifying potential issues
# There is a "problem" with Cross-checked tasks. Results appear multiple times and the only score we need is 
# the score in rows with "Cross-Check score" text in the 'comment' column. See an example below:
taskres_df.query('courseTaskId == 505 and studentId==14526')

,createdDate,updatedDate,score,comment,studentId,courseTaskId,historicalScores,juryScores,checkerId,anonymous
9015,2020-09-11 07:41:15.293934,2020-09-11 07:41:15.293934,153,Cross-Check score,14526,505,"[{""authorId"":-1,""score"":153,""dateTime"":1599810...",[],NaN,NaN
13292,2020-09-03 16:55:31.544966,2020-09-03 16:55:31.544966,230,Отличная работа!!!\nПравда имеется один значит...,14526,505,"[{""score"":230,""comment"":""Отличная работа!!!\nП...",NaN,15669.0,t
13431,2020-09-04 13:07:30.772535,2020-09-04 13:07:30.772535,230,При неверном ответе в окне справа должны появл...,14526,505,"[{""score"":230,""comment"":""При неверном ответе в...",NaN,14740.0,t
13901,2020-09-08 18:45:12.650121,2020-09-08 18:45:12.650121,0,Would you please give me the link to your pull...,14526,505,"[{""score"":0,""comment"":""Would you please give m...",NaN,14541.0,t


In [4]:
# STEP 3. Ommiting uninformative columns
task_df = task_df[['courseTaskId', 'courseId', 'maxScore', 'scoreWeight', 'taskId', 'type',  'checker']]
task_df

,courseTaskId,courseId,maxScore,scoreWeight,taskId,type,checker
0,296,12,40.0,1.000000,413,test,mentor
1,292,12,100.0,1.000000,253,NaN,mentor
2,288,12,120.0,1.000000,410,NaN,mentor
3,285,12,100.0,0.800000,407,NaN,mentor
4,289,12,150.0,1.000000,230,NaN,mentor
5,286,12,100.0,0.800000,408,NaN,mentor
6,284,12,120.0,0.300000,405,NaN,mentor
7,290,12,100.0,1.000000,411,NaN,mentor
8,297,12,100.0,0.600000,414,NaN,mentor
9,280,12,100.0,0.100000,404,NaN,mentor


In [5]:
taskres_df = taskres_df[['studentId', 'courseTaskId', 'score', 'comment']]
#taskres_df = taskres_df.query('courseTaskId != 343') 343 task is marked as 'trash' 
taskres_df

,studentId,courseTaskId,score,comment
0,7266,300,100,NaN
1,7929,313,41,NaN
2,6134,308,100,NaN
3,5882,305,100,NaN
4,8247,305,100,NaN
...,...,...,...,...
94257,14467,509,0,404 Error instead of application
94258,14457,509,68,Частично выполненные пункты:\n1) Есть переключ...
94259,14495,509,75,Выполненные пункты:\n1) Реализована генерация ...
94260,14444,483,170,Отзыв по пунктам ТЗ:\nНе выполненные/не засчит...


In [6]:
tr_df = tr_df[['studentId', 'courseId', 'mentorId', 'repository', 'isTopPerformer']]
tst_df = tst_df[['studentId', 'courseId', 'mentorId', 'repository']]

In [7]:
# STEP 4. Creating a new data frame that contains all the data related to tasks and students' scores.
# The output is a list of all tasks completed by every student, their scores, and task-related info
# Thus, we aim to 'enrich' the existing taskres_df table
tasks_df = pd.merge(left=taskres_df,right=task_df,how='left',on='courseTaskId',suffixes=('', ''))

tasks_df['score'] = tasks_df['score']*tasks_df['scoreWeight'] # get the 'real' Score
tasks_df['maxScore'] = tasks_df['maxScore']*tasks_df['scoreWeight'] # get the 'real' maxScore
tasks_df = tasks_df.drop('scoreWeight', axis=1) # remove scoreWeigth as it's not needed anymore
tasks_df

,studentId,courseTaskId,score,comment,courseId,maxScore,taskId,type,checker
0,7266,300,1.00,NaN,11,1.0,417,htmlcssacademy,mentor
1,7929,313,0.41,NaN,11,1.0,432,test,taskOwner
2,6134,308,2.00,NaN,11,2.0,426,jstask,mentor
3,5882,305,3.00,NaN,11,3.0,425,jstask,mentor
4,8247,305,3.00,NaN,11,3.0,425,jstask,mentor
...,...,...,...,...,...,...,...,...,...
94257,14467,509,0.00,404 Error instead of application,27,75.0,493,jstask,crossCheck
94258,14457,509,68.00,Частично выполненные пункты:\n1) Есть переключ...,27,75.0,493,jstask,crossCheck
94259,14495,509,75.00,Выполненные пункты:\n1) Реализована генерация ...,27,75.0,493,jstask,crossCheck
94260,14444,483,170.00,Отзыв по пунктам ТЗ:\nНе выполненные/не засчит...,27,300.0,452,jstask,crossCheck


In [8]:
# STEP 5. Overcoming the issue identified on STEP 2.

# I need to say I tried to calculate the mean of all other values so that the resulting data frame
# contains both "crossCheck" and the mean of other values grouped by studentId, but this approach turned out to be worse.
# I looks like students with only one cross-check score don't have their final cross-check score and therefore their score 
# should not be taken into account.

# create a dataframe with the 'real' crosscheck scores
cc_w_cc_comment = tasks_df.query('checker == "crossCheck" and comment == "Cross-Check score"')
print('comment == "Cross-Check score", count:', cc_w_cc_comment.shape[0])
# delete rows with the "crossCheck" value in the checker column
tasks_df = tasks_df.drop(tasks_df[tasks_df.checker == "crossCheck"].index) 
# add correct values with the real cross-check value
tasks_df = tasks_df.append(cc_w_cc_comment, ignore_index=True)

tasks_df

comment == "Cross-Check score", count: 6113


,studentId,courseTaskId,score,comment,courseId,maxScore,taskId,type,checker
0,7266,300,1.00,NaN,11,1.0,417,htmlcssacademy,mentor
1,7929,313,0.41,NaN,11,1.0,432,test,taskOwner
2,6134,308,2.00,NaN,11,2.0,426,jstask,mentor
3,5882,305,3.00,NaN,11,3.0,425,jstask,mentor
4,8247,305,3.00,NaN,11,3.0,425,jstask,mentor
...,...,...,...,...,...,...,...,...,...
70030,14518,509,68.00,Cross-Check score,27,75.0,493,jstask,crossCheck
70031,14506,509,75.00,Cross-Check score,27,75.0,493,jstask,crossCheck
70032,14492,509,32.00,Cross-Check score,27,75.0,493,jstask,crossCheck
70033,14495,509,75.00,Cross-Check score,27,75.0,493,jstask,crossCheck


In [9]:
# STEP 6. Getting the sum of all scores for each student in each course
course_student_score_df = tasks_df.groupby(['courseId', 'studentId']).sum()[['score']]
course_student_score_df

score
courseId studentId        
1        1          1324.4
         2           394.9
         3             0.0
         4           102.7
         5             2.4
...                    ...
29       15636       217.0
         15651        24.0
         15669       255.0
         15671       220.0
         15691       252.0

[8381 rows x 1 columns]

In [10]:
# STEP 7. Glue together two data frames as the test data frame may contain the best of the not-so-performing students 

# create a new isTest column so that we can easily extract test/ train data from the combined dataframe
tr_df['isTest'] = False 
tst_df['isTest'] = True
tst_df['isTopPerformer'] = 'UNKNOWN'
uni_df = pd.concat([tr_df,tst_df],ignore_index=True)
#uni_df.groupby(['courseId','isTopPerformer']).count()[['studentId']]
uni_df

,studentId,courseId,mentorId,repository,isTopPerformer,isTest
0,5740,11,647.0,https://github.com/rolling-scopes-school/1b3f6...,True,False
1,6956,11,795.0,https://github.com/rolling-scopes-school/76c0e...,True,False
2,11616,16,NaN,NaN,False,False
3,11773,16,NaN,NaN,False,False
4,6724,11,928.0,https://github.com/rolling-scopes-school/7fb23...,True,False
...,...,...,...,...,...,...
13457,13278,19,NaN,NaN,UNKNOWN,True
13458,14984,29,NaN,NaN,UNKNOWN,True
13459,14557,29,NaN,NaN,UNKNOWN,True
13460,15249,29,NaN,NaN,UNKNOWN,True


In [11]:
# STEP 8. Merging the list of task results and the list of students.
# Perform right join as there are students that are not in the task_results table

test_train_dataset = pd.merge(left=course_student_score_df, right=uni_df, \
                                  how='right',on=['studentId','courseId'],suffixes=('', ''))

print("The number of students with no score:", test_train_dataset.query('score != score').count()['courseId'])
print("The number of Top Performers with no score:", test_train_dataset.\
      query('score != score and isTopPerformer == True').count()['courseId'])

# substitute NAs with 0
test_train_dataset['score'] = test_train_dataset['score'].fillna(0)

test_train_dataset

The number of students with no score: 5081
The number of Top Performers with no score: 0


,studentId,courseId,score,mentorId,repository,isTopPerformer,isTest
0,5740,11,190.20,647.0,https://github.com/rolling-scopes-school/1b3f6...,True,False
1,6956,11,123.07,795.0,https://github.com/rolling-scopes-school/76c0e...,True,False
2,11616,16,0.00,NaN,NaN,False,False
3,11773,16,0.00,NaN,NaN,False,False
4,6724,11,400.62,928.0,https://github.com/rolling-scopes-school/7fb23...,True,False
...,...,...,...,...,...,...,...
13457,13278,19,0.00,NaN,NaN,UNKNOWN,True
13458,14984,29,0.00,NaN,NaN,UNKNOWN,True
13459,14557,29,0.00,NaN,NaN,UNKNOWN,True
13460,15249,29,0.00,NaN,NaN,UNKNOWN,True


In [12]:
# STEP 8. Displaying the percentage of each category (True/False/Unknown) grouped by courses + number of students 
# and their max and max scores so that we can understand how many values we need to  predict in each course.
number_of_students_in_groups = test_train_dataset.groupby(['courseId', 'isTopPerformer']).count()[['score']]
percentage_of_students_in_groups = number_of_students_in_groups.rename(columns={'score': 'Percentage'})
percentage_of_students_in_groups = round(100*(percentage_of_students_in_groups/percentage_of_students_in_groups.groupby(level=0).sum()),1)
number_of_students_in_groups = number_of_students_in_groups.rename(columns={'score': 'Count'})
max_scores_in_groups = test_train_dataset.groupby(['courseId', 'isTopPerformer']).max()[['score']]
max_scores_in_groups = max_scores_in_groups.rename(columns={'score': 'maxScore'})
min_scores_in_groups = test_train_dataset.groupby(['courseId', 'isTopPerformer']).min()[['score']]
min_scores_in_groups = min_scores_in_groups.rename(columns={'score': 'minScore'})
merged = pd.merge(number_of_students_in_groups, percentage_of_students_in_groups, left_index=True, right_index=True, suffixes=('', ''))
merged = pd.merge(merged, max_scores_in_groups, left_index=True, right_index=True, suffixes=('', ''))
merged = pd.merge(merged, min_scores_in_groups, left_index=True, right_index=True, suffixes=('', ''))
merged

Count  Percentage     maxScore     minScore
courseId isTopPerformer                                             
1        False            1199        65.0    105.80000     0.000000
         True              392        21.2   1589.40000   107.500000
         UNKNOWN           254        13.8    240.00000     0.000000
2        False             867        65.5    279.00000     0.000000
         True              277        20.9   1623.00000   283.000000
         UNKNOWN           179        13.5      0.00000     0.000000
4        False              63        64.9   1250.00000    71.000000
         True               20        20.6   2208.00000  1268.000000
         UNKNOWN            14        14.4      0.00000     0.000000
5        False              27        60.0   1829.00000     0.000000
         True               10        22.2   3520.18882  1844.314081
         UNKNOWN             8        17.8      0.00000     0.000000
6        False              26        61.9    386.00000     0.000000
         True                9        21.4    471.00000   409.000000
         UNKNOWN             7        16.7      0.00000     0.000000
9        False             257        64.9    803.00060     0.000000
         True               80        20.2   1391.00078   808.000320
         UNKNOWN            59        14.9      0.00000     0.000000
10       False              60        65.2    281.00000     0.000000
         True               18        19.6    861.80000   339.500000
         UNKNOWN            14        15.2      0.00000     0.000000
11       False            2016        64.6     54.00000     0.000000
         True              653        20.9   1834.71000    54.060000
         UNKNOWN           454        14.5   1354.70000     0.000000
12       False              14        66.7    462.70000   304.700000
         True                4        19.0    683.00000   492.700000
         UNKNOWN             3        14.3    648.70000   412.500000
13       False            2110        63.6     89.10000     0.000000
         True              697        21.0   1851.50000    89.680000
         UNKNOWN           513        15.5   1253.00000     0.000000
14       False              16        66.7    242.50000   164.501050
         True                5        20.8    329.50109   249.001070
         UNKNOWN             3        12.5    238.50000   194.001040
16       False             265        62.9    404.00000     0.000000
         True               80        19.0    803.00000   301.000000
         UNKNOWN            76        18.1    508.00000     0.000000
17       False              86        61.9    391.00000     0.000000
         True               29        20.9    716.00000   525.000000
         UNKNOWN            24        17.3    620.00000     0.000000
18       False             189        59.6    198.00000     0.000000
         True               69        21.8    752.00000   223.700000
         UNKNOWN            59        18.6    483.00000     0.000000
19       False             238        61.3   4440.00000     0.000000
         True               80        20.6  10440.00000  4450.000000
         UNKNOWN            70        18.0   2590.00000     0.000000
20       False             127        61.1    290.70000     0.000000
         True               45        21.6   1440.80000   315.000000
         UNKNOWN            36        17.3    889.30000     0.000000
21       False             289        63.5    398.00000     0.000000
         True               98        21.5    600.00000   394.000000
         UNKNOWN            68        14.9    500.00000     0.000000
23       False               1        33.3    200.00000   200.000000
         True                1        33.3    231.00000   231.000000
         UNKNOWN             1        33.3    170.00000   170.000000
27       False              42        67.7    368.00000     0.000000
         True               12        19.4    683.00000   342.000000
         UNKNOWN       

<hr>
We see that in most courses the number of unknown rows is not higher than 20 percents.<br>
The only problematic courses are 28 and 29.<br>
Also, we see that in the majority of cases the max score of non-performing students is lower than the min score of top performers. This gives us an idea this value (score) can properly divide the dataset and predict whether a student is a top performer or not.

In [13]:
# STEP 9. Calculating absolute scores.
# To do so, we'll find the best registered score in each score and divide scores of students by that value.

def get_max_registered_score(data, course_ids):
    max_scores = {}
    for course_id in course_ids:
        max_score = data.query('courseId == @course_id').max()['score']
        max_scores[course_id] = round(max_score,1)
    return max_scores

courseIDs = sorted(course_df['courseId'].unique())
max_scores = get_max_registered_score(test_train_dataset, courseIDs)

def calc_total_abs_score(row):
    course_id = row['courseId']
    course_max_score = max_scores[course_id]
    row['abs_score'] = row['score']/course_max_score
    return row

test_train_dataset['abs_score'] = 0

test_train_dataset['totalScore'] = test_train_dataset['score'].fillna(0)
test_train_dataset = test_train_dataset.apply(calc_total_abs_score, axis=1)
test_train_dataset[['mentorId', 'repository']] = test_train_dataset[['mentorId', 'repository']].fillna('N/A')
test_train_dataset = test_train_dataset.drop(columns='score')
test_train_dataset

,studentId,courseId,mentorId,repository,isTopPerformer,isTest,abs_score,totalScore
0,5740,11,647,https://github.com/rolling-scopes-school/1b3f6...,True,False,0.103668,190.20
1,6956,11,795,https://github.com/rolling-scopes-school/76c0e...,True,False,0.067079,123.07
2,11616,16,N/A,N/A,False,False,0.000000,0.00
3,11773,16,N/A,N/A,False,False,0.000000,0.00
4,6724,11,928,https://github.com/rolling-scopes-school/7fb23...,True,False,0.218357,400.62
...,...,...,...,...,...,...,...,...
13457,13278,19,N/A,N/A,UNKNOWN,True,0.000000,0.00
13458,14984,29,N/A,N/A,UNKNOWN,True,0.000000,0.00
13459,14557,29,N/A,N/A,UNKNOWN,True,0.000000,0.00
13460,15249,29,N/A,N/A,UNKNOWN,True,0.000000,0.00


In [14]:
# STEP 10*. There's an idea I'd like to check - what if we can predict whether a student is a Top Performer or not based
# on their absolute position in the leaderboard (i.e. top 20%)?


def set_howManyHasLowerScore(row):
    """
    This function calculates how many students (which proportiong of students) have a lower score
    Input: df row, output: df row with an extra 'howManyHasLowerScore' column
    """
    course = row['courseId']
    score = row['abs_score']
    course_dataset = test_train_dataset.query('courseId == @course')
    total_number_of_students = course_dataset.shape[0]
    number_of_worse_students = course_dataset[course_dataset['abs_score'] < score]['studentId'].count()
    the_proportion = number_of_worse_students/total_number_of_students
    row['howManyHasLowerScore'] = the_proportion
    return row
    
test_train_dataset['howManyHasLowerScore'] = 0
test_train_dataset = test_train_dataset.apply(set_howManyHasLowerScore, axis=1)
test_train_dataset

,studentId,courseId,mentorId,repository,isTopPerformer,isTest,abs_score,totalScore,howManyHasLowerScore
0,5740,11,647,https://github.com/rolling-scopes-school/1b3f6...,True,False,0.103668,190.20,0.852706
1,6956,11,795,https://github.com/rolling-scopes-school/76c0e...,True,False,0.067079,123.07,0.830612
2,11616,16,N/A,N/A,False,False,0.000000,0.00,0.000000
3,11773,16,N/A,N/A,False,False,0.000000,0.00,0.000000
4,6724,11,928,https://github.com/rolling-scopes-school/7fb23...,True,False,0.218357,400.62,0.881844
...,...,...,...,...,...,...,...,...,...
13457,13278,19,N/A,N/A,UNKNOWN,True,0.000000,0.00,0.000000
13458,14984,29,N/A,N/A,UNKNOWN,True,0.000000,0.00,0.000000
13459,14557,29,N/A,N/A,UNKNOWN,True,0.000000,0.00,0.000000
13460,15249,29,N/A,N/A,UNKNOWN,True,0.000000,0.00,0.000000


In [15]:
# STEP 11. It would be great to see all scores in course for each student, isn't it?
# So we'll pivot the data set, but firstly, we need to prepare the dataframe for pivoting

test_train_dataset_for_pivoting = pd.merge(left=test_train_dataset, right=tasks_df, \
                        how='left',on=['studentId','courseId'],suffixes=('', ''))
test_train_dataset_for_pivoting[['courseTaskId', 'taskId']] = test_train_dataset_for_pivoting[['courseTaskId', 'taskId']].fillna('N/A')
test_train_dataset_for_pivoting['score'] = test_train_dataset_for_pivoting['score'].fillna(0)
test_train_dataset_for_pivoting

,studentId,courseId,mentorId,repository,isTopPerformer,isTest,abs_score,totalScore,howManyHasLowerScore,courseTaskId,score,comment,maxScore,taskId,type,checker
0,5740,11,647,https://github.com/rolling-scopes-school/1b3f6...,True,False,0.103668,190.2,0.852706,319,30.0,NaN,30.0,439,jstask,mentor
1,5740,11,647,https://github.com/rolling-scopes-school/1b3f6...,True,False,0.103668,190.2,0.852706,302,2.0,NaN,2.0,423,jstask,mentor
2,5740,11,647,https://github.com/rolling-scopes-school/1b3f6...,True,False,0.103668,190.2,0.852706,304,5.0,NaN,5.0,424,jstask,mentor
3,5740,11,647,https://github.com/rolling-scopes-school/1b3f6...,True,False,0.103668,190.2,0.852706,318,0.5,NaN,1.0,437,cv:markdown,mentor
4,5740,11,647,https://github.com/rolling-scopes-school/1b3f6...,True,False,0.103668,190.2,0.852706,303,3.0,NaN,3.0,422,jstask,mentor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75111,13278,19,N/A,N/A,UNKNOWN,True,0.000000,0.0,0.000000,N/A,0.0,NaN,NaN,N/A,NaN,NaN
75112,14984,29,N/A,N/A,UNKNOWN,True,0.000000,0.0,0.000000,N/A,0.0,NaN,NaN,N/A,NaN,NaN
75113,14557,29,N/A,N/A,UNKNOWN,True,0.000000,0.0,0.000000,N/A,0.0,NaN,NaN,N/A,NaN,NaN
75114,15249,29,N/A,N/A,UNKNOWN,True,0.000000,0.0,0.000000,N/A,0.0,NaN,NaN,N/A,NaN,NaN


In [16]:
# STEP 12. Now it's time to pivot the data.
# Once pivoted csv files are generated, it's worth checking the generated csv files to see if there any visible outliers

courseId_studentId_isTest_score_howmany = pd.DataFrame()
pivoted_courses_collection = {}

for courseID in courseIDs:
    print('Pivoting', courseID)
    course_df = test_train_dataset_for_pivoting.query('courseId == @courseID')
    pivoted_course_df = pd.pivot_table(course_df, index=['courseId','studentId', 'isTopPerformer', 'isTest', 'abs_score', \
                                                        'howManyHasLowerScore', 'totalScore', 'mentorId', 'repository'], \
                               columns=['courseTaskId'], values=['score'], fill_value=0)
    flat_pivoted_course_df = pivoted_course_df
    # make the pivot table flat
    flat_pivoted_course_df.columns = ['_'.join([str(c) for c in c_list]) for c_list in flat_pivoted_course_df.columns.values]
    pivoted_courses_collection[courseID] = flat_pivoted_course_df
    filename = str(courseID)+'_pivoted_data.csv'
    # to ease the process of reading the data, we'll sort the rows by students' scores
    flat_pivoted_course_df_ordered = flat_pivoted_course_df.sort_values(by='abs_score', ascending=False)
    flat_pivoted_course_df_ordered.to_csv(path_or_buf='pivoted_tables/'+filename, encoding='utf-8-sig')
    courseId_studentId_isTest_score_howmany = pd.concat([courseId_studentId_isTest_score_howmany, \
                                              flat_pivoted_course_df_ordered.reset_index()\
                                              [['courseId', 'studentId', 'abs_score', 'howManyHasLowerScore', 'isTopPerformer', 'isTest']]],\
                                             ignore_index=True)

courseId_studentId_isTest_score_howmany

Pivoting 1
Pivoting 2
Pivoting 4
Pivoting 5
Pivoting 6
Pivoting 9
Pivoting 10
Pivoting 11
Pivoting 12
Pivoting 13
Pivoting 14
Pivoting 16
Pivoting 17
Pivoting 18
Pivoting 19
Pivoting 20
Pivoting 21
Pivoting 23
Pivoting 27
Pivoting 28
Pivoting 29


,courseId,studentId,abs_score,howManyHasLowerScore,isTopPerformer,isTest
0,1,490,1.000000,0.999458,True,False
1,1,415,0.968101,0.998916,True,False
2,1,316,0.966025,0.998374,True,False
3,1,149,0.948408,0.997832,True,False
4,1,513,0.947150,0.997290,True,False
...,...,...,...,...,...,...
13457,29,15064,0.000000,0.000000,UNKNOWN,True
13458,29,15067,0.000000,0.000000,UNKNOWN,True
13459,29,15068,0.000000,0.000000,UNKNOWN,True
13460,29,15070,0.000000,0.000000,UNKNOWN,True


In [17]:
# STEP 13. PREDICTING
# At first, let's try the decision tree (0.86657 public score)
# I made an assumption that can be considered stupid that courses 28 and 29 can be predicted based on absolute scores
# in similar courses.
# It's still not something I would use in the real world, but that's what turned out to give the best result 
# (I guess, it happened just because of the high number of false positives)

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score


student_predictedPerformance = pd.DataFrame()

for courseID in courseIDs:
    course_data = courseId_studentId_isTest_score_howmany.query('courseId == @courseID')
    print('Building the decision tree:', courseID)
    if courseID in [23,28,29]: # course 28 is trained on 17 and course 29 is trained on 16 as these courses are similar
        if courseID == 23: # as course 23 has a very little data, we'll train the model on all other (but 28,29) courses
            train = courseId_studentId_isTest_score_howmany.query('isTest == False')[['studentId', 'abs_score', 'isTopPerformer']]
            test = course_data.query('isTest == True')[['studentId', 'abs_score']]
            X_train, y_train  = train[['abs_score']], train['isTopPerformer'].astype(int)
            X_test = test[['abs_score']]
        # this is the moment when courses.csv comes in handy. We see that course 28 and 17 are similar
        elif courseID == 28: 
            train = courseId_studentId_isTest_score_howmany.query('isTest == False and courseId == 17')[['studentId', 'abs_score', 'isTopPerformer']]
            test = course_data.query('isTest == True')[['studentId', 'abs_score']]
            X_train, y_train  = train[['abs_score']], train['isTopPerformer'].astype(int)
            X_test = test[['abs_score']]
        elif courseID == 29:
            train = courseId_studentId_isTest_score_howmany.query('isTest == False and courseId == 16')[['studentId', 'abs_score', 'isTopPerformer']]
            test = course_data.query('isTest == True')[['studentId', 'abs_score']]
            X_train, y_train  = train[['abs_score']], train['isTopPerformer'].astype(int)
            X_test = test[['abs_score']]
    else:
        train = course_data.query('isTest == False')[['studentId', 'abs_score', 'isTopPerformer']]
        test = course_data.query('isTest == True')[['studentId', 'abs_score']]
        X_train, y_train  = train[['abs_score']], train['isTopPerformer'].astype(int)
        X_test = test[['abs_score']]
    # No grid searching is here as we have only one incoming column and it's not expected to have some complex logic.
    # I have to admit I tried tuning some hyperparameters just to prove I'm not wrong
    DTC = DecisionTreeClassifier(criterion='gini', max_depth=1, 
                                 min_samples_split=2, 
                                 min_samples_leaf=1, random_state=0) 
    DTC.fit(X_train, y_train)
    y_pred = DTC.predict(X_test)
    test['isTopPerformer'] = y_pred.astype(bool)
    #print(test)
    student_predictedPerformance = pd.concat([student_predictedPerformance, test], ignore_index=True)    
    

tst_ordered_template = pd.read_csv("data/test.csv") # read the dataframe 
tst_ordered_template = tst_ordered_template[['studentId']] # extract the column so that the prediction has the same order
# restore the order of rows according to the test file
res = pd.merge(tst_ordered_template,student_predictedPerformance,how='left', on='studentId') 
res = res.reset_index()
res[['studentId', 'isTopPerformer']].to_csv(path_or_buf='output/task8_decision_tree.csv', index=False)
res

Building the decision tree: 1
Building the decision tree: 2
Building the decision tree: 4
Building the decision tree: 5
Building the decision tree: 6
Building the decision tree: 9
Building the decision tree: 10
Building the decision tree: 11
Building the decision tree: 12
Building the decision tree: 13
Building the decision tree: 14
Building the decision tree: 16
Building the decision tree: 17
Building the decision tree: 18
Building the decision tree: 19
Building the decision tree: 20
Building the decision tree: 21
Building the decision tree: 23
Building the decision tree: 27
Building the decision tree: 28
Building the decision tree: 29


,index,studentId,abs_score,isTopPerformer
0,0,11450,0.546700,True
1,1,7855,0.056500,True
2,2,12753,0.009579,False
3,3,14717,0.000000,False
4,4,7455,0.643053,True
...,...,...,...,...
2986,2986,13278,0.000000,False
2987,2987,14984,0.000000,False
2988,2988,14557,0.000000,False
2989,2989,15249,0.000000,False


In [18]:
# STEP 14. Now it's time to use some sophisticated algorithm! 
# If-else, for example ;)  (0.86968)

# get max non-top and min top scores in each course
def get_abs_threshold(data, course_ids):
    thresholds = {}
    for course_id in course_ids[:-2]:
        max_false_score = data.query('courseId == @course_id and isTopPerformer == False').max()['abs_score']
        min_true_score = data.query('courseId == @course_id and isTopPerformer == True').min()['abs_score']
        thresholds[course_id] = [max_false_score, min_true_score]
    return thresholds

abs_thresholds = get_abs_threshold(test_train_dataset, courseIDs)
print(abs_thresholds)

# here goes the algo.
def predict(row):
    courseID = row['courseId']
    if courseID not in [28,29]:
        # here max_false_score and min_true_score are compared; the condition is > max_false_score or
        # >= min_true_score
        if abs_thresholds[courseID][1] > abs_thresholds[courseID][0]:
            row['isTopPerformer'] = 'True' if row['abs_score'] > abs_thresholds[courseID][0] else 'False'
        else:
            row['isTopPerformer'] = 'True' if row['abs_score'] >= abs_thresholds[courseID][1] else 'False'
    elif courseID == 28:
        if abs_thresholds[17][1] > abs_thresholds[17][0]:
            row['isTopPerformer'] = 'True' if row['abs_score'] > abs_thresholds[17][0] else 'False'
        else:
            row['isTopPerformer'] = 'True' if row['abs_score'] >= abs_thresholds[17][1] else 'False'
    elif courseID == 29:
        if abs_thresholds[16][1] > abs_thresholds[16][0]:
            row['isTopPerformer'] = 'True' if row['abs_score'] > abs_thresholds[16][0] else 'False'
        else:
            row['isTopPerformer'] = 'True' if row['abs_score'] >= abs_thresholds[16][1] else 'False'
    return row

res = pd.read_csv("data/test.csv").reset_index()[['studentId']]
test_dataset = courseId_studentId_isTest_score_howmany[['courseId', 'studentId', 'abs_score']]
res = pd.merge(res,test_dataset,how='left', on='studentId') # restore the order of rows according to the test file
res = res.apply(predict, axis=1)
res = res.reset_index()
res['studentId'] = res['studentId'].astype(int)
res[['studentId', 'isTopPerformer']].to_csv(path_or_buf='output/task8_if_else.csv', index=False)
res

{1: [0.0665659997483327, 0.06763558575563106], 2: [0.17190388170055454, 0.17436845348120764], 4: [0.5661231884057971, 0.5742753623188406], 5: [0.5195727515482075, 0.5239230955627522], 6: [0.8195329087048833, 0.8683651804670913], 9: [0.5772829618979152, 0.5808772969086987], 10: [0.3260617312601532, 0.39394291018797867], 11: [0.02943260478552352, 0.029465307679729658], 12: [0.6774524158125915, 0.7213762811127379], 13: [0.04812314339724547, 0.04843640291655415], 14: [0.7359635811836115, 0.7556936874051593], 16: [0.5031133250311333, 0.37484433374844334], 17: [0.5460893854748603, 0.7332402234636871], 18: [0.2632978723404255, 0.29747340425531915], 19: [0.42528735632183906, 0.42624521072796934], 20: [0.20176290949472514, 0.21862853970016657], 21: [0.6633333333333333, 0.6566666666666666], 23: [0.8658008658008658, 1.0], 27: [0.5387994143484627, 0.5007320644216691]}


,index,studentId,courseId,abs_score,isTopPerformer
0,0,11450,16.0,0.546700,True
1,1,7855,11.0,0.056500,True
2,2,12753,19.0,0.009579,False
3,3,14717,28.0,0.000000,False
4,4,7455,11.0,0.643053,True
...,...,...,...,...,...
2986,2986,13278,19.0,0.000000,False
2987,2987,14984,29.0,0.000000,False
2988,2988,14557,29.0,0.000000,False
2989,2989,15249,29.0,0.000000,False


In [19]:
# STEP 15. While it's algo scored less that two above, I believe it's still the best one

student_predictedPerformance = pd.DataFrame()

for courseID in courseIDs:
    course_data = courseId_studentId_isTest_score_howmany.query('courseId == @courseID')
    print('Building decision tree:', courseID)
    courseIDs_no2829 = courseIDs[:-2]
    if courseID in [23,28,29]:
        if courseID == 23:
            train = courseId_studentId_isTest_score_howmany.query('isTest == False')[['studentId', 'abs_score', 'isTopPerformer']]
            test = course_data.query('isTest == True')[['studentId', 'abs_score']]
            X_train, y_train  = train[['abs_score']], train['isTopPerformer'].astype(int)
            X_test = test[['abs_score']]
        # this time let's try to train the model for courses 28 and 29 based on the all available data 
        elif courseID in [28, 29]:
            train = courseId_studentId_isTest_score_howmany.query('isTest == False and courseId == @courseIDs_no2829')[['studentId', 'howManyHasLowerScore', 'isTopPerformer']]
            print(train.shape)
            test = course_data.query('isTest == True')[['studentId', 'howManyHasLowerScore']]
            X_train, y_train  = train[['howManyHasLowerScore']], train['isTopPerformer'].astype(int)
            X_test = test[['howManyHasLowerScore']]
    else:
        train = course_data.query('isTest == False')[['studentId', 'abs_score', 'isTopPerformer']]
        test = course_data.query('isTest == True')[['studentId', 'abs_score']]
        X_train, y_train  = train[['abs_score']], train['isTopPerformer'].astype(int)
        X_test = test[['abs_score']]
    DTC = DecisionTreeClassifier(criterion='gini', max_depth=1, 
                                 min_samples_split=2, 
                                 min_samples_leaf=1, random_state=0)
    DTC.fit(X_train, y_train)
    y_pred = DTC.predict(X_test)
    test['isTopPerformer'] = y_pred.astype(bool)
    student_predictedPerformance = pd.concat([student_predictedPerformance, test], ignore_index=True)    

tst_ordered_template = pd.read_csv("data/test.csv")
tst_ordered_template = tst_ordered_template[['studentId']]
res = pd.merge(tst_ordered_template,student_predictedPerformance,how='left', on='studentId') # restore the order of rows according to the test file
res = res.reset_index()
res[['studentId', 'isTopPerformer']].to_csv(path_or_buf='output/task8_decision_tree_with_howManyHasLowerScore.csv', index=False)
res

Building decision tree: 1
Building decision tree: 2
Building decision tree: 4
Building decision tree: 5
Building decision tree: 6
Building decision tree: 9
Building decision tree: 10
Building decision tree: 11
Building decision tree: 12
Building decision tree: 13
Building decision tree: 14
Building decision tree: 16
Building decision tree: 17
Building decision tree: 18
Building decision tree: 19
Building decision tree: 20
Building decision tree: 21
Building decision tree: 23
Building decision tree: 27
Building decision tree: 28
(10471, 3)
Building decision tree: 29
(10471, 3)


,index,studentId,abs_score,isTopPerformer,howManyHasLowerScore
0,0,11450,0.546700,True,NaN
1,1,7855,0.056500,True,NaN
2,2,12753,0.009579,False,NaN
3,3,14717,NaN,False,0.0
4,4,7455,0.643053,True,NaN
...,...,...,...,...,...
2986,2986,13278,0.000000,False,NaN
2987,2987,14984,NaN,False,0.0
2988,2988,14557,NaN,False,0.0
2989,2989,15249,NaN,False,0.0


<hr>
Conclusion: life is hard but it's quite exciting<br>
On a serious note, I'm surprised why no one scored higher than 87%.<br>
Most likely, some score shouldn't be taken into account or there is a lack of data for course 29 (some courses are missing).<br>
The other idea is that the proportion off the 'top' students is vastly different from other courses